# Directly create h5 from aligned next fasta.
Also has functions to process to variant fasta

In [3]:
import numpy as np
import os as os
import sys as sys
import multiprocessing as mp
import pandas as pd
import socket
import matplotlib.pyplot as plt
from itertools import groupby
from shutil import which
import os
import re as re
import h5py

### Pick the right path (whether on cluster or at home)
socket_name = socket.gethostname()
print(f"Current machine: {socket_name}")
if socket_name == "DESKTOP-5RJD9NC":
    path = "/gitProjects/covid19_data"   # The Path on Harald's machine
if socket_name.startswith("compute-"):
    print("HSM Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/covid19_data/"  # The Path on Midway Cluster
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")
    
### Check whether required bins are available
req_bins = ["mafft"] 
for b in req_bins:
    s = which(b)
    if not s:
        print(f"Make sure to install {b} and have in path. I cannot find it!")
        
os.chdir(path)  # Set the right Path (in line with Atom default)
print(os.getcwd())

sys.path.append("./python3/")
from manipulate_fasta import fasta_iter_raw, fasta_iter

Current machine: compute-a-16-106.o2.rc.hms.harvard.edu
HSM Computational partition detected.
/n/groups/reich/hringbauer/git/covid19_data


In [62]:
def save_h5(seqs, iids, path, output=True):
        """Create a new HDF5 File with Sequence Data.
        Each sequence entry is saved as a single char.
        seqs: Genomic Data: [k,l] array
        iids: IIDs to save into HDF5.
        path: Where to save the HDF5 to"""
        k, l = np.shape(seqs)  # Nr of Individuals and Nr of Loci
        assert(k == len(iids)) ## Sanity Chec
        
        max_len = np.max([len(iid) for iid in iids]) # Longest iid

        if os.path.exists(path):  # Do a Deletion of existing File there
            os.remove(path)
            
        dt = h5py.special_dtype(vlen=str)  # To have no problem with saving

        with h5py.File(path, 'w') as f0:
            ### Create all the Groups
            f_gt = f0.create_dataset("gt", (k,l), dtype='|S1')
            f_samples = f0.create_dataset("samples", (k,), dtype=dt)

            ### Save the Data
            f_gt[:] = seqs.astype("|S1")
            f_samples[:] = np.array(iids)   #.astype("S" + max_len)

        if output == True:
            print(f"Successfully saved {k} individuals to: {path}")
            
def get_iids_seqs_from_aligned_fasta(path_fasta="./data/oct20/sequences_2020-10-30_07-23.fasta"):
    """Get IIDs and Sequences from aligned Fasta file."""

    fiter = fasta_iter(path_fasta)
    print("Getting IIDs...")
    iids = np.array([ff[0] for ff in fiter])

    fiter = fasta_iter(path_fasta)
    s = next(fiter)[1]

    seqs = np.empty((len(iids),len(s)), dtype="|S1")
    fiter = fasta_iter(path_fasta)
    for i in range(len(iids)):
        if i%10000==0:
            print(f"Running sequence {i}")
        s = next(fiter)[1]
        seqs[i,:]=list(s)
    return iids, seqs

def post_process_counts(ct, drop=[b"n", b"-"]):
    """Post-process Pandas value Counts.
    drop: What allele to drop and not to count"""
    ct = ct.drop(labels = drop, errors="ignore")
    if len(ct)==1:
        gt_ct = [ct.values[0], 0]
        alleles = [ct.index[0], ""]
    else:
        gt_ct = [ct.values[0], ct.values[1]]
        alleles = [ct.index[0], ct.index[1]]
    return gt_ct, alleles

def produce_allele_stats(seq, drop=[b"n", b"-"], loci=[],
                         decode=True):
    """Produce allele counts for major/minor allele
    loci: Which Loci to analyze
    decode: Whether to decode S1 strings in output"""
    counts = []
    
    if len(loci)==0:
        k = np.shape(seqs)[1]
        loci = range(0,k)
        print(f"Running all Loci: n={len(loci)}")
        
    ref_count, alt_count = np.zeros(len(loci), dtype="int"), np.zeros(len(loci), dtype="int")
    ref, alt  =  np.empty(len(loci), dtype="|S1"), np.empty(len(loci), dtype="|S1")
    
    for i,l in enumerate(loci):
        ct = pd.value_counts(seqs[:,l])
        gts_c, alleles = post_process_counts(ct, drop=drop)
        ref_count[i], alt_count[i] = gts_c
        ref[i], alt[i] = alleles
    
    df = pd.DataFrame({"refcount":ref_count, "altcount":alt_count, 
                  "ref":ref, "alt":alt, "pos":loci})
    if decode:
        df['ref'] = df['ref'].str.decode("utf-8")
        df['alt'] = df['alt'].str.decode("utf-8")
    return df

def extract_var_df(df, frac_tot=0.75, maf=0.05):
    """Extract and return Dataframe with MAF > maf.
    frac_tot: Minimum total individuals with coverage thre"""
    tot_count = df1["altcount"] + df1["refcount"]
    count_good =  tot_count>(frac_tot*np.max(tot_count))
    print(f"Minimum Count per Locus required: {frac_tot*np.max(tot_count)}")
    idx = (df1["altcount"] / tot_count > maf) & count_good

    print(f"Found {np.sum(idx)} Variants with MAF>{maf}")
    df_var = df1[idx].copy()

    df_var["totcount"] = df_var["refcount"] + df_var["altcount"]
    df_var["maf"] = df_var["altcount"]/df_var["totcount"]
    return df_var

# Get IIDs and Sequences

In [63]:
iids, seqs = get_iids_seqs_from_aligned_fasta(path_fasta="./data/oct20/sequences_2020-10-30_07-23.fasta")

Getting IIDs...
Running sequence 0


ValueError: cannot copy sequence with size 29896 to array axis with dimension 29891

### Save them to HDF5

In [11]:
%%time
save_h5(seqs, iids, path="./output/h5/covid_seqs_next.h5")
#seqs = np.array([ff[1] for ff in fiter])

Successfully saved 166919 individuals to: ./output/h5/covid_seqs_next.h5
CPU times: user 2.44 s, sys: 7.75 s, total: 10.2 s
Wall time: 2min 14s


### Create Allele Frequency Spectrum

In [52]:
ct = pd.value_counts(seqs[1008,:])
ct

b'G'    29891
dtype: int64

In [ ]:
pd.val

In [14]:
%%time
df1 = produce_allele_stats(seqs, loci=range(23400,23410), drop=[b"-", b"n"])

CPU times: user 427 ms, sys: 15.8 ms, total: 443 ms
Wall time: 440 ms


In [15]:
df1

,refcount,altcount,ref,alt,pos
0,124638,18226,A,C,23400
1,124638,18226,A,C,23401
2,124638,18226,A,C,23402
3,124638,18226,A,C,23403
4,124638,18226,A,C,23404
5,124638,18226,A,C,23405
6,124638,18226,A,C,23406
7,124638,18226,A,C,23407
8,124638,18226,A,C,23408
9,124638,18226,A,C,23409


In [27]:
savepath = "./output/tables/allele_spectrum_next.tsv"
df1.to_csv(savepath, sep="\t", index=False)
print(f"Saved {len(df1)} Loci Statistics to {savepath}")

Saved 29891 Loci Statistics to ./output/tables/allele_spectrum_next.tsv


# Create the table of Variants

In [4]:
%%time
savepath = "./output/tables/allele_spectrum_next.tsv"
df1 = pd.read_csv(savepath, sep="\t")
print(f"Loaded {len(df1)} Loci Statistics from {savepath}")
mx_ref = np.max(df1["refcount"])
print(f"Max Ref Count: {mx_ref}")

Loaded 29891 Loci Statistics from ./output/tables/allele_spectrum_next.tsv
Max Ref Count: 124638
CPU times: user 19.6 ms, sys: 9.97 ms, total: 29.5 ms
Wall time: 31.3 ms


In [5]:
### Showcase what variants we have
df_var = extract_var_df(df1, maf = 0.01, frac_tot = 0.4)

### Which Mutations
mutations = df_var["ref"] + ">" + df_var["alt"]
mutations.value_counts()

Minimum Count per Locus required: 57145.600000000006
Found 29891 Variants with MAF>0.01


A>C    29891
dtype: int64

In [6]:
df1

,refcount,altcount,ref,alt,pos
0,124638,18226,A,C,0
1,124638,18226,A,C,1
2,124638,18226,A,C,2
3,124638,18226,A,C,3
4,124638,18226,A,C,4
...,...,...,...,...,...
29886,124638,18226,A,C,29886
29887,124638,18226,A,C,29887
29888,124638,18226,A,C,29888
29889,124638,18226,A,C,29889


# Area 51

In [3]:
path_fasta = "./data/oct20/sequences_2020-10-30_07-23.fasta"
fiter = fasta_iter(path_fasta)

In [ ]:
next(fiter)

In [ ]:
print("finished!")

In [ ]:
ll = np.array([len(s) for s in seqs])

### Test filling empty strings

In [54]:
seqs = np.empty((3,2), dtype="|S1")

In [64]:
seqs[1,:]=list("abc")

ValueError: cannot copy sequence with size 3 to array axis with dimension 2

In [61]:
seqs

array([[b'\xc8', b'7'],
       [b'a', b'b'],
       [b'\xd9', b'\x7f']], dtype='|S1')